In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import requests
import xmltodict
import time

In [2]:
authKey = "G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D"

## 전체 아파트 목록 조회 함수
def get_apt_list(authKey = authKey):
    url = f"http://apis.data.go.kr/1613000/AptListService2/getTotalAptList?serviceKey={authKey}&pageNo=1&numOfRows=1000000"
    response = requests.get(url)
    raw = xmltodict.parse(response.text)
    raw = pd.DataFrame(raw['response']['body']['items']['item'])
    return raw

## 아파트 기본 정보 조회 함수
def get_danji_base_info(danji_cd, authKey = authKey):
    url = f"http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo?serviceKey={authKey}&kaptCode={danji_cd}"
    response = requests.get(url)
    raw = xmltodict.parse(response.text)
    raw = pd.DataFrame(raw['response']['body']['item'], index=[0])
    return raw

## 아파트 상세 정보 조회 함수
def get_danji_detail_info(danji_cd, authKey = authKey):
    url = f"http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusDtlInfo?serviceKey={authKey}&kaptCode={danji_cd}"
    response = requests.get(url)
    raw = xmltodict.parse(response.text)
    raw = pd.DataFrame(raw['response']['body']['item'], index=[0])
    return raw

In [3]:
danji_list = get_apt_list()
danji_list.columns = ['시도명', '시군구명', '읍면동명', '리명', '법정동코드', '단지코드', '아파트명']

In [6]:
danji_cd_ls = danji_list[danji_list['시도명'] == "서울특별시"]['단지코드'].to_list()

In [7]:
## 1차 획득
_ls = []
for danji_cd in danji_cd_ls:
    try:
        _ls.append(get_danji_base_info(danji_cd))
    except Exception as e:
        print(f"{danji_cd} : {e}")
base_info = pd.concat(_ls)

A13982604 : 'response'
A15179701 : 'response'
A12009001 : HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /1613000/AptBasisInfoService1/getAphusBassInfo?serviceKey=G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D&kaptCode=A12009001 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000298E95D54D0>, 'Connection to apis.data.go.kr timed out. (connect timeout=None)'))
A15086202 : 'response'
A15004406 : HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /1613000/AptBasisInfoService1/getAphusBassInfo?serviceKey=G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D&kaptCode=A15004406 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000298E9ACEE50>, 'Connection to apis.data.go.kr timed out. (connect timeout=None)'))
A12201103 : 'response'
A10088102 : HTTPConnectionPool(host='apis.

In [23]:
## 오류 부분 획득
err_ls = []
for danji_cd in [code for code in danji_cd_ls if code not in base_info['kaptCode'].to_list()]:
    try:
        err_ls.append(get_danji_base_info(danji_cd))
    except Exception as e:
        print(f"{danji_cd} : {e}")
base_info_err = pd.concat(err_ls)

In [30]:
## 저장
path = 'C:/Users/GCC/git'
base_info_res = pd.concat([base_info, base_info_err])
base_info_res.to_csv(f"{path}/data/DL/아파트기본정보.csv", index=False)

In [34]:
## 이거 해야 됨
_ls = []
for danji_cd in danji_cd_ls:
    try:
        _ls.append(get_danji_detail_info(danji_cd))
    except Exception as e:
        print(f"{danji_cd} : {e}")
detail_info = pd.concat(_ls)

A13527007 : HTTPConnectionPool(host='apis.data.go.kr', port=80): Read timed out. (read timeout=None)
A13202308 : HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /1613000/AptBasisInfoService1/getAphusDtlInfo?serviceKey=G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D&kaptCode=A13202308 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000298E9F4C790>, 'Connection to apis.data.go.kr timed out. (connect timeout=None)'))
A15809402 : HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /1613000/AptBasisInfoService1/getAphusDtlInfo?serviceKey=G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D&kaptCode=A15809402 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000298EA49F7D0>, 'Connection to apis.data.go.kr timed out. (connect timeout=None)'))
A12009001 : HTTPConnectionPool(host

In [35]:
## 오류 부분 획득
err_ls = []
for danji_cd in [code for code in danji_cd_ls if code not in detail_info['kaptCode'].to_list()]:
    try:
        err_ls.append(get_danji_detail_info(danji_cd))
    except Exception as e:
        print(f"{danji_cd} : {e}")
detail_info_err = pd.concat(err_ls)

In [36]:
## 저장
path = 'C:/Users/GCC/git'
detail_info_res = pd.concat([detail_info, detail_info_err])
detail_info_res.to_csv(f"{path}/data/DL/아파트상세정보.csv", index=False)